# (Generalized) Linear and Hierarchical Linear Models in PyMC3

In [ ]:
import arviz as az
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pymc3 as pm
import statsmodels.api as sm
import theano
import bambi
from numpy.random import default_rng
from pandas.plotting import scatter_matrix
from pymc3 import *
from statsmodels.formula.api import glm as glm_sm

print(f"Running on PyMC3 v{pm.__version__}")

In [ ]:
%config InlineBackend.figure_format = 'retina'
# Initialize random number generator
RANDOM_SEED = 8927
rng = default_rng(RANDOM_SEED)
az.style.use("arviz-darkgrid")

## Linear Regression

Lets generate some data with known slope and intercept and fit a simple linear GLM.

In [ ]:
size = 50
true_intercept = 1
true_slope = 2
x = np.linspace(0, 1, size)
y = true_intercept + x * true_slope + rng.normal(scale=0.5, size=size)
data = {"x": x, "y": y}

The `glm.linear_component()` function can be used to generate the output variable y_est and coefficients of the specified linear model.

In [ ]:
with Model() as model:
    lm = glm.LinearComponent.from_formula("y ~ x", data)
    sigma = Uniform("sigma", 0, 20)
    y_obs = Normal("y_obs", mu=lm.y_est, sigma=sigma, observed=y)
    trace = pm.sample(2000, cores=2, return_inferencedata=True)

plt.figure(figsize=(5, 5))
plt.plot(x, y, "x")
plot_posterior_predictive_glm(trace)

Since there are a couple of general linear models that are being used over and over again (Normally distributed noise, logistic regression etc), the `glm.glm()` function simplifies the above step by creating the likelihood (y_obs) and its priors (sigma) for us. Since we are working in the model context, the random variables are all added to the model behind the scenes. This function also automatically finds a good starting point which it returns.

Note that the below call to `glm()` is producing exactly the same model as above, just more succinctly.

In [ ]:
with Model() as model:
    GLM.from_formula("y ~ x", data)
    trace = pm.sample(2000, cores=2, return_inferencedata=True)

plt.figure(figsize=(5, 5))
plt.plot(x, y, "x")
plot_posterior_predictive_glm(trace)

## Robust GLM

Lets try the same model but with a few outliers in the data.

In [ ]:
x_out = np.append(x, [0.1, 0.15, 0.2])
y_out = np.append(y, [8, 6, 9])
data_outlier = dict(x=x_out, y=y_out)

In [ ]:
with Model() as model:
    GLM.from_formula("y ~ x", data_outlier)
    trace = pm.sample(2000, cores=2, return_inferencedata=True)

plt.figure(figsize=(5, 5))
plt.plot(x_out, y_out, "x")
plot_posterior_predictive_glm(trace)

Because the normal distribution does not have a lot of mass in the tails, an outlier will affect the fit strongly.

Instead, we can replace the Normal likelihood with a student T distribution which has heavier tails and is more robust towards outliers. While this could be done with the `linear_compoment()` function and manually defining the T likelihood we can use the `glm()` function for more automation. By default this function uses a normal likelihood. To define the usage of a T distribution instead we can pass a family object that contains information on how to link the output to `y_est` (in this case we explicitly use the Identity link function which is also the default) and what the priors for the T distribution are. Here we fix the degrees of freedom `nu` to 1.5.

In [ ]:
with Model() as model_robust:
    family = glm.families.StudentT(
        link=glm.families.Identity(), priors={"nu": 1.5, "lam": Uniform.dist(0, 20)}
    )
    GLM.from_formula("y ~ x", data_outlier, family=family)
    trace = pm.sample(2000, cores=2, return_inferencedata=True)

plt.figure(figsize=(5, 5))
plt.plot(x_out, y_out, "x")
plot_posterior_predictive_glm(trace)

## Hierarchical GLM

In [ ]:
try:
    sat_data = pd.read_csv("../data/Guber1999data.txt")
except:
    sat_data = pd.read_csv(pm.get_data("Guber1999data.txt"))

In [ ]:
grp_mean = bambi.Prior("Normal", mu=0, sigma=10)
grp_sd = bambi.Prior('HalfCauchy', beta=5)

priors = {'Intercept': bambi.Prior('Normal',mu=sat_data.sat_t.mean(), sigma=sat_data.sat_t.std()),
          '1|spend': bambi.Prior('Normal', mu=grp_mean, sigma=grp_sd),
          '1|stu_tea_rat':bambi.Prior('Normal',mu=grp_mean, sigma=grp_sd), 
          '1|salary': bambi.Prior('Normal', mu=grp_mean, sigma=grp_sd),
          '1|prcnt_take':bambi.Prior('Normal', mu=grp_mean, sigma=grp_sd)}


In [ ]:
model = bambi.Model("sat_t ~ (1|spend) + (1|stu_tea_rat) + (1|salary) + (1|prcnt_take)", sat_data,priors=priors)

In [ ]:
results = model.fit(draws=1000)

In [ ]:
scatter_matrix(results.to_dataframe, figsize=(12, 12));

In [ ]:
with Model() as model_sat:
    grp_mean = Normal("grp_mean", mu=0, sigma=10)
    grp_prec = Gamma("grp_prec", alpha=1, beta=0.1, testval=1.0)
    slope = StudentT.dist(mu=grp_mean, lam=grp_prec, nu=1)
    intercept = Normal.dist(mu=sat_data.sat_t.mean(), sigma=sat_data.sat_t.std())
    GLM.from_formula(
        "sat_t ~ spend + stu_tea_rat + salary + prcnt_take",
        sat_data,
        priors={"Intercept": intercept, "Regressor": slope},
    )
    trace_sat = sample(2000, cores=2)

In [ ]:
scatter_matrix(trace_to_dataframe(trace_sat), figsize=(12, 12));

In [ ]:
tdf_gain = 5.0
with Model() as model_sat:
    grp_mean = Normal("grp_mean", mu=0, sigma=10)
    grp_prec = Gamma("grp_prec", alpha=1, beta=0.1, testval=1.0)
    slope = StudentT.dist(mu=grp_mean, lam=grp_prec, nu=1)  # grp_df)
    intercept = Normal.dist(mu=sat_data.sat_t.mean(), sigma=sat_data.sat_t.std())
    GLM.from_formula(
        "sat_t ~ spend + stu_tea_rat + salary + prcnt_take",
        sat_data,
        priors={"Intercept": intercept, "Regressor": slope},
    )

    trace_sat = sample(2000, cores=2)

In [ ]:
scatter_matrix(trace_to_dataframe(trace_sat), figsize=(12, 12));

## Logistic Regression

In [ ]:
htwt_data = pd.read_csv(get_data("HtWt.csv"))
htwt_data.head()

In [ ]:
m = glm_sm("male ~ height + weight", htwt_data, family=sm.families.Binomial()).fit()
print(m.summary())

In [ ]:
with Model() as model_htwt:
    GLM.from_formula("male ~ height + weight", htwt_data, family=glm.families.Binomial())
    trace_htwt = sample(
        2000, cores=2, init="adapt_diag"
    )  # default init with jitter can cause problem

In [ ]:
trace_df = trace_to_dataframe(trace_htwt)
print(trace_df.describe().drop("count").T)
scatter_matrix(trace_df, figsize=(8, 8))
print("P(weight < 0) = ", (trace_df["weight"] < 0).mean())
print("P(height < 0) = ", (trace_df["height"] < 0).mean())

## Bayesian Logistic Lasso

In [ ]:
lp = Laplace.dist(mu=0, b=0.05)
x_eval = np.linspace(-0.5, 0.5, 300)
plt.plot(x_eval, theano.tensor.exp(lp.logp(x_eval)).eval())
plt.xlabel("x")
plt.ylabel("Probability")
plt.title("Laplace distribution");

In [ ]:
with Model() as model_lasso:
    # Define priors for intercept and regression coefficients.
    priors = {"Intercept": Normal.dist(mu=0, sigma=50), "Regressor": Laplace.dist(mu=0, b=0.05)}
    GLM.from_formula(
        "male ~ height + weight", htwt_data, family=glm.families.Binomial(), priors=priors
    )

    trace_lasso = sample(500, cores=2, init="adapt_diag")

trace_df = trace_to_dataframe(trace_lasso)
scatter_matrix(trace_df, figsize=(8, 8))
print(trace_df.describe().drop("count").T)

In [ ]:
%load_ext watermark
%watermark -n -u -v -iv -w